In [47]:
import pandas as pd 
import numpy as np 
import pandas_profiling
import datetime

#Original file 
csv_path = '/Users/chrisrios/Desktop/Data_science/airbnb_post_merge/DS4A/austindata.csv'

#Making the Dataframe
trulia_df = pd.read_csv(csv_path)

#output the dataframe
print(trulia_df)

#rename column with no space 
trulia_df.rename(columns = {'fullLocation':'full_location', 'partialLocation':'street_address','price':'home_price','bedrooms':'num_bedrooms' ,'bathrooms':'num_bathrooms'}, inplace = True)

#Drop columns that are irrelevent/redundant and not found in other data sources
trulia_df.drop(columns= ['ID','File','stateCode'],inplace= True)


      ID                                               File    city stateCode  \
0      0  <_io.TextIOWrapper name='/Users/bhava/Desktop/...  Austin        TX   
1      1  <_io.TextIOWrapper name='/Users/bhava/Desktop/...  Austin        TX   
2      2  <_io.TextIOWrapper name='/Users/bhava/Desktop/...  Austin        TX   
3      3  <_io.TextIOWrapper name='/Users/bhava/Desktop/...  Austin        TX   
4      4  <_io.TextIOWrapper name='/Users/bhava/Desktop/...  Austin        TX   
...   ..                                                ...     ...       ...   
1226  26  <_io.TextIOWrapper name='/Users/bhava/Desktop/...  Austin        TX   
1227  27  <_io.TextIOWrapper name='/Users/bhava/Desktop/...  Austin        TX   
1228  28  <_io.TextIOWrapper name='/Users/bhava/Desktop/...  Austin        TX   
1229  29  <_io.TextIOWrapper name='/Users/bhava/Desktop/...  Austin        TX   
1230  30  <_io.TextIOWrapper name='/Users/bhava/Desktop/...  Austin        TX   

      zipCode              

In [48]:
"""
Drop all cities that arent Austin 
    -everything after drop is the criteria
    -df.index selects 
"""
trulia_df.drop(trulia_df[trulia_df['city'] != "Austin"].index, inplace=True)

In [51]:
#Cleaning up the coordinates into two seperate columns
for index,row in trulia_df.iterrows():
    split_coordinates = trulia_df.loc[index,'coordinates'].split(',')
    trulia_df.loc[index, 'latitude'] = split_coordinates[0].replace("{'latitude': ","")
    trulia_df.loc[index, 'longitude'] = split_coordinates[1].replace(" 'longitude': ","")

#Cleaning sqft
#for index,row in trulia_df.iterrows():
    #split_coordinates = trulia_df.loc[index,'sqft'].split(',')
    #trulia_df.loc[index, 'latitude'] = split_coordinates[0].replace("{'formattedDimension': '","")

trulia_df['floorSpace'].replace("N/A",np.nan)


#for index,row in trulia_df.iterrows():
    #split_sqft = trulia_df.loc[index,'floorSpace'].split(' ')
    #trulia_df.loc[index, 'sqft'] = split_sqft[1].replace(",","")




0       {'formattedDimension': '2,013 sqft', '__typena...
1       {'formattedDimension': '2,274 sqft', '__typena...
2       {'formattedDimension': '2,235 sqft', '__typena...
3       {'formattedDimension': '2,981 sqft', '__typena...
4       {'formattedDimension': '1,518 sqft', '__typena...
                              ...                        
1226                                                  NaN
1227    {'formattedDimension': '2,104 sqft', '__typena...
1228    {'formattedDimension': '1,643 sqft', '__typena...
1229    {'formattedDimension': '976 sqft', '__typename...
1230    {'formattedDimension': '2,600 sqft', '__typena...
Name: floorSpace, Length: 1172, dtype: object

In [ ]:

trulia_df.drop(columns= ['coordinates','city'],inplace= True)

In [ ]:
#Merge CSVs (Original and ZipCode one)

In [38]:
#Check Data Types
trulia_df.dtypes

city              object
zipCode            int64
full_location     object
street_address    object
coordinates       object
home_price        object
sqft              object
num_bedrooms      object
num_bathrooms     object
latitude          object
longitude         object
dtype: object

In [ ]:
#Change repetitive data into Categories (efficiency)
def convertToCategory(*args):
    for col in args:
        trulia_df[col] = trulia_df[col].astype('category')

def convertToInt(*args):
    for col in args:
        trulia_df[col] = trulia_df[col].astype('int64')

#Convert dates to Datetime DT
def convertToDt(*args):
    for arg in args:
        trulia_df[arg] = pd.to_datetime(trulia_df[arg])

convertToCategory('city','home_type')
convertToDt('latest_saledate','latest_salemonth', 'latest_saleyear')

trulia_df.dtypes

In [ ]:
#Home Image Transformation
#Remove end string (-p_f.jpg) 
trulia_df['home_image'] = trulia_df['home_image'].str.replace("-p_f.jpg","")

#Strip off everything up to _ from images for url creation
for index,row in trulia_df.iterrows():
      #Grabbing the original string
      try: 
            partition_string = trulia_df.loc[index , 'home_image'].partition("_")
      except AttributeError:
            print("There was an AttributeError somewhere")
            
      else:    
            trulia_df.loc[index ,'home_image'] = partition_string[2]

In [ ]:
trulia_df.describe()

In [ ]:
#Change Home image to include full URL 
#Grabbed strings from the img request when visiting site
default_image_urlbeg = 'https://www.trulia.com/pictures/thumbs_5/zillowstatic/fp/'
default_image_urlend = '-full.webp'

#apply the URL schema via a new column
trulia_df['home_imageURL'] = trulia_df['home_image'].apply(lambda x: default_image_urlbeg + str(x) + default_image_urlend)

#Remove all invalid home images that have a long length 
for index,row in trulia_df.iterrows():
   if len( row['home_imageURL']) > 125:
      trulia_df.loc[index ,'home_imageURL'] = "Not Available"

#Drop the original homeImage column 
trulia_df.drop(columns = 'home_image', inplace=True)

In [ ]:
#Trim Blank Space in Description column
trulia_df['description'] = trulia_df['description'].str.strip()
trulia_df['latest_price'] = trulia_df['latest_price'].str.strip()

def replaceCharacters(*args, column, newvalue):
    """
    Replaces x strings of characters, takes column and the new value to replace the values with 
    """
    for oldvalue in args:
        trulia_df[column] = trulia_df[column].replace(oldvalue,newvalue)

#Replace missing values in Description Column 
replaceCharacters("","-",".","....",'\\', column='description', newvalue="Not Available")

#drop $ and commas from home values
trulia_df['latest_price'] = trulia_df['latest_price'].str.replace("$","")
trulia_df['latest_price'] = trulia_df['latest_price'].str.replace(",","")
trulia_df['latest_price'] = trulia_df["latest_price"].astype(float).astype(int)

#convertToInt('latest_price')

In [ ]:
#Pull out the rent from Description 
substring = "The Rent Zestimate for this home is $"

#Grab the estimated rent from the description
for index, row in trulia_df.iterrows():
    rent_string = trulia_df.loc[index, 'description'].partition(substring)
    #If the estimate wasnt null append it to the corresponding index and column
    if rent_string[2] != "":
        trulia_df.loc[index,'rent_estimate'] = rent_string[2]

#Get rid of non numeric values

trulia_df['rent_estimate'] = trulia_df['rent_estimate'].str.replace("/mo.","")
trulia_df['rent_estimate'] = trulia_df['rent_estimate'].str.replace(",","")

#Change Nan to 0's
trulia_df['rent_estimate'] = trulia_df['rent_estimate'].fillna(0)

In [ ]:
#Dealing with outliers

def changePropertyValue(address, column, newValue):
    """
    Parameters: property address, column you want to change value in, the New value
    """
    trulia_df.loc[trulia_df.street_address == address, column] = newValue

changePropertyValue('4212 Far West Blvd', 'num_bathrooms', 3)
changePropertyValue('1812 Eagles Glen Cv', 'rent_estimate', 5600)
changePropertyValue('1812 Eagles Glen Cv', 'latest_price', 999000)
changePropertyValue('7109 Greenshores Dr', 'rent_estimate', 7865)
changePropertyValue('7109 Greenshores Dr', 'latest_price', 1125000)
changePropertyValue('1907 Holly St', 'latest_price', 2000000)
changePropertyValue('1500 Hartford Rd', 'latest_price', 2149100)
changePropertyValue('1500 Hartford Rd', 'rent_estimate', 6417)
changePropertyValue('3602 River Rd', 'latest_price', 1990000)
changePropertyValue('1604 Eva St', 'latest_price', 1799900)
changePropertyValue('1604 Eva St', 'rental_estimate',  7100)
changePropertyValue('2800 Westlake Dr', 'latest_price', 1995000)
changePropertyValue('8801 Young Ln', 'latest_price', 1049900 )
changePropertyValue('4109 E 12th St #2','latest_price',675400)

#Deleting outliers that arent true or arent viable properties
trulia_df.drop(trulia_df.loc[trulia_df['num_bedrooms'] > 15].index, inplace=True)
trulia_df.drop(trulia_df.loc[trulia_df['num_bedrooms'] == 0].index, inplace=True)
trulia_df.drop(trulia_df.loc[trulia_df['num_bathrooms'] == 0].index, inplace=True)
trulia_df.drop(trulia_df.loc[trulia_df['home_type'] == 'Vacant Land'].index, inplace=True)


In [ ]:
def generate_Report():
    #Generate the profile Report
    profile = pandas_profiling.ProfileReport(trulia_df, title = "Pandas Profiling Report") 
    #Display within the notebook
    profile.to_notebook_iframe()

